In [1]:
FILE_LIST = 'files.p'
DATA_DIR = "/run/media/maciej/Nowy/data/json/"
CHOSEN_YEAR = "2011"

In [14]:
import os, json, pickle, sys, os, glob, time
from collections import defaultdict
from urllib.request import urlopen, Request

from tqdm import tqdm
import regex



In [3]:
def get_first_k_judgments(k=100):
    files = pickle.load(open(FILE_LIST, 'rb'))

    total_judgments = []
    for file in tqdm(files):
        if file.startswith("judgment"):
            file_path = os.path.join(DATA_DIR, file)

            with open(file_path, 'r') as f:
                data = json.load(f)
                judgments = [
                    (x["id"], x["judgmentDate"]) for x in data["items"] 
                    if x["judgmentDate"].startswith(CHOSEN_YEAR)
                ]
            total_judgments += judgments

    sorted_by_dates = sorted(total_judgments, key=lambda x: x[1])
    ids, _ = zip(*sorted_by_dates)
    
    return ids[:k]

first_100 = get_first_k_judgments()

100%|██████████| 68/68 [00:01<00:00, 35.76it/s]


In [4]:
word_pattern = "\p{Letter}+"

def load_data(ids):
    total_judgments = []
    files = pickle.load(open(FILE_LIST, 'rb'))
    
    for file in files:
        if file.startswith("judgment"):
            file_path = os.path.join(DATA_DIR, file)

            with open(file_path, 'r') as f:
                data = json.load(f)
                judgments = [
                    x["textContent"] for x in data["items"] 
                    if x["id"] in ids
                ]
                
            total_judgments += judgments
            
    filtered_judgments = []
    for judgment in tqdm(total_judgments):
        judgment = regex.sub("<.*?>", "", judgment)
        judgment = regex.sub("-\n(\p{Letter}+)", r"\1", judgment)
        judgment = judgment.lower()
        
        filtered_judgments.append(judgment)
        
    
    return filtered_judgments

judgments = load_data(first_100)

100%|██████████| 100/100 [00:00<00:00, 2751.14it/s]


In [5]:
import requests

r = requests.get("http://ws.clarin-pl.eu/nlprest2/base/upload/Ala ma kota z Warszawy")

In [6]:
r.text

'/users/default/8db5ddbf-6a6c-4fc7-a8e5-1a5d94574c50'

In [8]:
url = "http://ws.clarin-pl.eu/nlprest2/base/download/users/default/3bebc1bc-a225-4fe3-95e1-4bd187c7aa78"
requests.get(url).text


'Ala ma kota z Warszawy'

any2txt|wcrft2({"morfeusz2":false})|iobber|liner2({"model":"n82","output":"tei:gz"})|spejd


In [40]:
#!/usr/bin/python
# -*- coding: utf-8 -*-


url = "http://ws.clarin-pl.eu/nlprest2/base"


def download(id):
    content = urlopen(Request(url + '/download' + id)).read()
    print(content)
    return content


def process(lpmn, data):

    data = {
        'lpmn': lpmn,
        'text': data
    }

    import requests
    taskid = requests.post(
        url + '/startTask/', 
        json.dumps(data), 
        {'Content-Type': 'application/json'},
    ).text

    time.sleep(0.05)

    resp = urlopen(Request(url + '/getStatus/' + taskid))

    data = json.load(resp)

    while data["status"] == "QUEUE" or data["status"] == "PROCESSING":
        time.sleep(1)
        resp = urlopen(Request(url + '/getStatus/' + taskid))
        data = json.load(resp)
    
    if data["status"] == "ERROR":
        print("Error " + data["value"])
        return

    return data["value"][0]['fileID']

In [41]:
lpmn = 'any2txt|wcrft2({"morfeusz2":false})|iobber|liner2({"model":"n82","output":"tei:gz"})|spejd'

for judgment in tqdm(judgments):
    file_id = process(lpmn, judgment)
    download(file_id)
    break


  0%|          | 0/100 [00:00<?, ?it/s]
Exception in thread Thread-14:
Traceback (most recent call last):
  File "/usr/lib64/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/maciej/.virtualenvs/nlp/lib/python3.6/site-packages/tqdm/_tqdm.py", line 148, in run
    for instance in self.tqdm_cls._instances:
  File "/home/maciej/.virtualenvs/nlp/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration




HTTPError: HTTP Error 500: Internal Server Error

In [43]:
file_id

'/requests/spejd/62f5a4ff-3a0a-4609-8da7-bea73ba9b5a0'

In [46]:
download(file_id)

HTTPError: HTTP Error 500: Internal Server Error